### importing Libraries

*   pandas for working with tabular data (tables).
*   numpy for working with numeric values such as the special NaN value that represents "Not a Number".

In [1]:
import numpy as np
import pandas as pd

### Loading the dataset

Loading dataset sensor_log.csv contains readings from a temperature and humidity sensor from the file in google colab

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('sensor_log.csv')

# Look at the first 5 rows
df.head()


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
2,2025-10-01 08:00:20,24.6,55.1,NaN
3,2025-10-01 08:00:30,NaN,54.9,3.68
4,2025-10-01 08:01:00,24.9,54.8,3.68


In [6]:
# How many rows and columns does the dataset have?
print('Number of rows:', df.shape[0])
print('Number of columns:', df.shape[1])

# General information about the DataFrame, including data types and non-null counts
df.info()

Number of rows: 10
Number of columns: 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      10 non-null     object 
 1   temperature_c  8 non-null      float64
 2   humidity_pct   9 non-null      float64
 3   voltage_v      9 non-null      float64
dtypes: float64(3), object(1)
memory usage: 452.0+ bytes


### Exercise 1 – Missing values in `temperature_c` and `humidity_pct`



In [3]:

# 1. Rows where temperature_c is missing
if "temperature_c" in df.columns:
    temp_missing = df[df["temperature_c"].isna()]
    print("Rows where temperature_c is missing:")
    display(temp_missing.head())
    print(f"Total rows with missing temperature_c: {temp_missing.shape[0]}")
else:
    print("Column 'temperature_c' not found in the dataset.")

# 2. Rows where humidity_pct is missing
if "humidity_pct" in df.columns:
    humidity_missing = df[df["humidity_pct"].isna()]
    print("\nRows where humidity_pct is missing:")
    display(humidity_missing.head())
    print(f"Total rows with missing humidity_pct: {humidity_missing.shape[0]}")
else:
    print("Column 'humidity_pct' not found in the dataset.")

# 3. Column with the highest percentage of missing values
missing_counts = df.isna().sum()
missing_pct = missing_counts / len(df) * 100

print("\nMissing value counts per column:")
display(missing_counts.to_frame("missing_count"))

print("\nMissing value percentages per column:")
display(missing_pct.to_frame("missing_percent"))

col_with_max_missing = missing_pct.idxmax()
print(f"\nColumn with the highest percentage of missing values: {col_with_max_missing} "
      f"({missing_pct[col_with_max_missing]:.2f}% missing)")


Rows where temperature_c is missing:


,timestamp,temperature_c,humidity_pct,voltage_v
3,2025-10-01 08:00:30,NaN,54.9,3.68
8,2025-10-01 08:08:00,NaN,55.0,3.64


Total rows with missing temperature_c: 2

Rows where humidity_pct is missing:


,timestamp,temperature_c,humidity_pct,voltage_v
5,2025-10-01 08:02:15,25.1,NaN,3.67


Total rows with missing humidity_pct: 1

Missing value counts per column:


,missing_count
timestamp,0
temperature_c,2
humidity_pct,1
voltage_v,1



Missing value percentages per column:


,missing_percent
timestamp,0.0
temperature_c,20.0
humidity_pct,10.0
voltage_v,10.0



Column with the highest percentage of missing values: temperature_c (20.00% missing)


For this exercise, the key idea is that we:
- Filtered the DataFrame using `.isna()` to see only rows where a specific column is missing.
- Computed missing counts and percentages for **every** column.
- Identified the column with the highest fraction of missing values as the one that may need special attention (dropping or careful imputation).

### Exercise 2 – Median imputation vs mean imputation


In [4]:
# Create copies for mean and median based imputation
df_mean = df.copy()
df_median = df.copy()

# Identify numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", numeric_cols)

# Mean-based imputation
for col in numeric_cols:
    df_mean[col] = df_mean[col].fillna(df_mean[col].mean())

# Median-based imputation
for col in numeric_cols:
    df_median[col] = df_median[col].fillna(df_median[col].median())

print("\nPreview of original data (with missing values):")
display(df[numeric_cols].head())

print("\nPreview after MEAN-based imputation:")
display(df_mean[numeric_cols].head())

print("\nPreview after MEDIAN-based imputation:")
display(df_median[numeric_cols].head())


Numeric columns: ['temperature_c', 'humidity_pct', 'voltage_v']

Preview of original data (with missing values):


,temperature_c,humidity_pct,voltage_v
0,24.5,55.2,3.70
1,24.7,55.0,3.69
2,24.6,55.1,NaN
3,NaN,54.9,3.68
4,24.9,54.8,3.68



Preview after MEAN-based imputation:


,temperature_c,humidity_pct,voltage_v
0,24.500,55.2,3.700000
1,24.700,55.0,3.690000
2,24.600,55.1,3.667778
3,25.075,54.9,3.680000
4,24.900,54.8,3.680000



Preview after MEDIAN-based imputation:


,temperature_c,humidity_pct,voltage_v
0,24.5,55.2,3.70
1,24.7,55.0,3.69
2,24.6,55.1,3.67
3,25.0,54.9,3.68
4,24.9,54.8,3.68


**Discussion:**

- **Mean imputation** uses the arithmetic average. It is **sensitive to outliers**: very high or very low values can pull the mean up or down.
- **Median imputation** uses the middle value of the sorted data. It is **much more robust to extreme values**, because outliers do not change the median as dramatically.
- For real-world sensor or clinical data (which often contain occasional spikes or faulty readings), median-based imputation is usually the safer, more robust choice.

### Exercise 3 – Forward fill, backward fill, and interpolation


In [13]:

# Ensure we are working with a time-indexed DataFrame
df_ts = df.copy()
df_ts['timestamp'] = pd.to_datetime(df_ts['timestamp'])
df_ts = df_ts.set_index('timestamp')

print("df_ts index type:", type(df_ts.index))

# Create filled / interpolated versions
df_ffill = df_ts.ffill()          # forward fill
df_bfill = df_ts.bfill()          # backward fill
df_interp = df_ts.interpolate(method="time") if isinstance(df_ts.index, pd.DatetimeIndex) else df_ts.interpolate()

# Choose a small sample range to compare side by side.
# We'll take the first 15 rows (you can adjust this slice if you want).
sample_slice = df_ts.iloc[:15]

comparison = pd.DataFrame(index=sample_slice.index)
for col in sample_slice.columns:
    comparison[col] = sample_slice[col]
    comparison[f"{col}_ffill"] = df_ffill.loc[sample_slice.index, col]
    comparison[f"{col}_bfill"] = df_bfill.loc[sample_slice.index, col]
    comparison[f"{col}_interp"] = df_interp.loc[sample_slice.index, col]

print("Comparison of original vs filled/interpolated values (first 15 rows):")
display(comparison)


df_ts index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Comparison of original vs filled/interpolated values (first 15 rows):


,temperature_c,temperature_c_ffill,temperature_c_bfill,temperature_c_interp,humidity_pct,humidity_pct_ffill,humidity_pct_bfill,humidity_pct_interp,voltage_v,voltage_v_ffill,voltage_v_bfill,voltage_v_interp
timestamp,,,,,,,,,,,,
2025-10-01 08:00:00,24.5,24.5,24.5,24.500000,55.2,55.2,55.2,55.2000,3.70,3.70,3.70,3.700
2025-10-01 08:00:10,24.7,24.7,24.7,24.700000,55.0,55.0,55.0,55.0000,3.69,3.69,3.69,3.690
2025-10-01 08:00:20,24.6,24.6,24.6,24.600000,55.1,55.1,55.1,55.1000,NaN,3.69,3.68,3.685
2025-10-01 08:00:30,NaN,24.6,24.9,24.675000,54.9,54.9,54.9,54.9000,3.68,3.68,3.68,3.680
2025-10-01 08:01:00,24.9,24.9,24.9,24.900000,54.8,54.8,54.8,54.8000,3.68,3.68,3.68,3.680
2025-10-01 08:02:15,25.1,25.1,25.1,25.100000,NaN,54.8,54.7,54.7375,3.67,3.67,3.67,3.670
2025-10-01 08:03:00,25.3,25.3,25.3,25.300000,54.7,54.7,54.7,54.7000,3.67,3.67,3.67,3.670
2025-10-01 08:05:30,25.5,25.5,25.5,25.500000,54.9,54.9,54.9,54.9000,3.65,3.65,3.65,3.650
2025-10-01 08:08:00,NaN,25.5,26.0,25.777778,55.0,55.0,55.0,55.0000,3.64,3.64,3.64,3.640


**Discussion:**

- **Forward fill (ffill)** copies the last observed value forward in time. It works well when the true value does not change too quickly and when short gaps appear.
- **Backward fill (bfill)** copies the next observed value backward. This can sometimes introduce future information into the past and may be less realistic for time-based processes.
- **Interpolation** (especially time-based interpolation) estimates values smoothly between known points, which can be very reasonable for continuous sensor measurements like temperature or humidity.

For this sensor dataset, **time-based interpolation** or a combination of **forward fill** for small gaps and interpolation for larger ones is usually more realistic than pure backward fill.

## Final Practice – End-to-end missing values handling on `sensor_log.csv`



In [12]:

# 1. Load sensor_log.csv again as a fresh DataFrame
df_full = pd.read_csv(csv_path)

print("First 5 rows of the raw dataset:")
display(df_full.head())

print("\nDataset shape (rows, columns):", df_full.shape)

# 2. Summarise missing values per column
missing_counts = df_full.isna().sum()
missing_pct = missing_counts / len(df_full) * 100

summary_missing = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_percent": missing_pct
}).sort_values("missing_percent", ascending=False)

print("\nMissing values summary:")
display(summary_missing)

# 3. Decide which columns/rows to drop
# Example rule: drop columns with more than 40% missing values
high_missing_cols = summary_missing[summary_missing["missing_percent"] > 40].index.tolist()
print("\nColumns with > 40% missing values (candidate for dropping):", high_missing_cols)

df_reduced = df_full.drop(columns=high_missing_cols)

print("\nShape after dropping high-missing columns:", df_reduced.shape)

# Optionally, drop rows with too many missing values (e.g., more than 50% of remaining columns missing)
row_missing_frac = df_reduced.isna().mean(axis=1)
rows_to_keep = row_missing_frac <= 0.5
df_reduced_rows = df_reduced[rows_to_keep]

print("\nShape after also dropping rows with > 50% missing across remaining columns:", df_reduced_rows.shape)

# 4. Choose and apply an imputation strategy for the remaining missing values
import numpy as np

numeric_cols = df_reduced_rows.select_dtypes(include=[np.number]).columns.tolist()
non_numeric_cols = [c for c in df_reduced_rows.columns if c not in numeric_cols]

print("\nNumeric columns to impute:", numeric_cols)
print("Non-numeric columns to impute (if any):", non_numeric_cols)

# Strategy:
# - For numeric columns: use MEDIAN (robust to outliers)
# - For categorical columns: use MODE (most frequent value)
df_clean = df_reduced_rows.copy()

# Median for numeric
for col in numeric_cols:
    median_value = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_value)

# Mode for non-numeric (categorical) columns
for col in non_numeric_cols:
    if df_clean[col].isna().any():
        mode_value = df_clean[col].mode(dropna=True)
        if not mode_value.empty:
            df_clean[col] = df_clean[col].fillna(mode_value.iloc[0])

print("\nCheck remaining missing values after imputation:")
display(df_clean.isna().sum())

print("\nPreview of final cleaned dataset:")
display(df_clean.head())

# Optionally, save the cleaned dataset to a new CSV file
output_clean_path = "sensor_log_cleaned.csv"
df_clean.to_csv(output_clean_path, index=False)

print(f"\nCleaned dataset saved to: {output_clean_path}")

First 5 rows of the raw dataset:


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
2,2025-10-01 08:00:20,24.6,55.1,NaN
3,2025-10-01 08:00:30,NaN,54.9,3.68
4,2025-10-01 08:01:00,24.9,54.8,3.68



Dataset shape (rows, columns): (10, 4)

Missing values summary:


,missing_count,missing_percent
temperature_c,2,20.0
humidity_pct,1,10.0
voltage_v,1,10.0
timestamp,0,0.0



Columns with > 40% missing values (candidate for dropping): []

Shape after dropping high-missing columns: (10, 4)

Shape after also dropping rows with > 50% missing across remaining columns: (10, 4)

Numeric columns to impute: ['temperature_c', 'humidity_pct', 'voltage_v']
Non-numeric columns to impute (if any): ['timestamp']

Check remaining missing values after imputation:


,0
timestamp,0
temperature_c,0
humidity_pct,0
voltage_v,0



Preview of final cleaned dataset:


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
2,2025-10-01 08:00:20,24.6,55.1,3.67
3,2025-10-01 08:00:30,25.0,54.9,3.68
4,2025-10-01 08:01:00,24.9,54.8,3.68



Cleaned dataset saved to: sensor_log_cleaned.csv


**Summary of decisions in the final practice:**

- We **inspected** missing values per column using counts and percentages.
- We **dropped columns** with very high missingness (more than 40%) because they provide too little usable information.
- We **dropped rows** that had more than half of their remaining values missing, since they would need too much guessing.
- We used **median imputation** for numeric columns to be robust to outliers.
- We used **mode imputation** for categorical columns so that missing values follow the most common existing category.
- Finally, we saved a cleaned version of the dataset as `sensor_log_cleaned.csv`, which can now be used for further analysis, visualisation, or machine learning.